In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [15]:
from rich.console import Console

console = Console()

In [23]:
dataPath = Path.cwd().parent / "tests" / "data"

plierRes_b = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "plierRes_b.csv", index_col=0
)
plierRes_c = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "plierRes_c.csv", index_col=0
)
plierRes_residual = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "plierRes_residual.csv", index_col=0
)
plierRes_u = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "plierRes_u.csv", index_col=0
)
plierRes_z = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "plierRes_z.csv", index_col=0
)

plierRes_uauc = pd.read_csv(
    filepath_or_buffer=dataPath / "getAUC" / "aucresults_uauc.csv", index_col=0
)
plierRes_up = pd.read_csv(
    filepath_or_buffer=dataPath / "getAUC" / "aucresults_upval.csv", index_col=0
)
plierRes_summary = pd.read_csv(
    filepath_or_buffer=dataPath / "getAUC" / "aucresults_summary.csv", index_col=0
)

priorMat = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "priorMat.csv", index_col=0
)
priorMatCV = pd.read_csv(
    filepath_or_buffer=dataPath / "common" / "priorMatCV.csv", index_col=0
)

plierRes = {
    "B": plierRes_b,
    "C": plierRes_c,
    "residual": plierRes_residual,
    "U": plierRes_u,
    "Z": plierRes_z,
    "Uauc": plierRes_uauc,
    "Up": plierRes_up,
    "summary": plierRes_summary,
}

In [24]:
top = 1
fdr_cutoff = 0.01
use = "coef"

In [25]:
if use is None:
    use = "coef"
elif use not in ("coef", "AUC"):
    raise ValueError(
        "only 'coef' and 'AUC' are the only valid options for the 'use' argument"
    )

In [26]:
if use == "coef":
    Uuse = plierRes["U"]
else:
    Uuse = plierRes["Uauc"]

In [27]:
plierRes["summary"]

,pathway,LV index,AUC,p-value,FDR
0,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY,LV1,0.642935,5.979941e-16,1.232230e-15
0,IRIS_Monocyte-Day0,LV2,0.771498,6.281456e-36,4.745989e-35
0,IRIS_Neutrophil-Resting,LV2,0.917797,6.403945e-126,4.354682e-124
0,SVM Neutrophils,LV2,0.942809,4.964885e-31,2.597017e-30
0,REACTOME_SIGNALLING_BY_NGF,LV4,0.593512,6.194152e-06,7.262109e-06
...,...,...,...,...,...
0,KEGG_PARKINSONS_DISEASE,LV28,0.835713,4.672248e-29,2.118086e-28
0,MIPS_SPLICEOSOME,LV28,0.606528,1.125169e-05,1.275191e-05
0,MIPS_55S_RIBOSOME_MITOCHONDRIAL,LV28,0.881966,3.253850e-29,1.580441e-28
0,KEGG_SPLICEOSOME,LV28,0.660846,7.638110e-10,1.038783e-09


In [28]:
plierRes["summary"].loc[plierRes["summary"]["FDR"] < fdr_cutoff]

,pathway,LV index,AUC,p-value,FDR
0,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY,LV1,0.642935,5.979941e-16,1.232230e-15
0,IRIS_Monocyte-Day0,LV2,0.771498,6.281456e-36,4.745989e-35
0,IRIS_Neutrophil-Resting,LV2,0.917797,6.403945e-126,4.354682e-124
0,SVM Neutrophils,LV2,0.942809,4.964885e-31,2.597017e-30
0,REACTOME_SIGNALLING_BY_NGF,LV4,0.593512,6.194152e-06,7.262109e-06
...,...,...,...,...,...
0,KEGG_PARKINSONS_DISEASE,LV28,0.835713,4.672248e-29,2.118086e-28
0,MIPS_SPLICEOSOME,LV28,0.606528,1.125169e-05,1.275191e-05
0,MIPS_55S_RIBOSOME_MITOCHONDRIAL,LV28,0.881966,3.253850e-29,1.580441e-28
0,KEGG_SPLICEOSOME,LV28,0.660846,7.638110e-10,1.038783e-09


In [29]:
if plierRes["Up"] is not None:
    pval_cutoff = max(
        plierRes["summary"].loc[plierRes["summary"]["FDR"] < fdr_cutoff, "p-value"]
    )
    Uuse[plierRes["Up"] > pval_cutoff] = 0
else:
    console("[red]No p-values in PLIER object: using coefficients only[/]")

In [33]:
mm = Uuse.apply(func=np.max, axis=0)

In [34]:
mm

LV1     0.029389
LV2     0.283651
LV3     0.000000
LV4     0.012788
LV5     0.148766
LV6     0.121878
LV7     0.022963
LV8     0.215824
LV9     0.115966
LV10    0.006628
LV11    0.142699
LV12    0.000000
LV13    0.071067
LV14    0.000000
LV15    0.176278
LV16    0.000000
LV17    0.000000
LV18    0.000000
LV19    0.000000
LV20    0.281207
LV21    0.096558
LV22    0.000000
LV23    0.182000
LV24    0.000000
LV25    0.000000
LV26    0.000000
LV27    0.191005
LV28    0.118849
LV29    0.001381
LV30    0.000000
dtype: float64

In [32]:
Uuse

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV21,LV22,LV23,LV24,LV25,LV26,LV27,LV28,LV29,LV30
index,,,,,,,,,,,,,,,,,,,,,
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
IRIS_Bcell-Memory_IgM,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
IRIS_Bcell-naive,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
IRIS_CD4Tcell-N0,0.0,0.0,0,0.0,0.0,0.031636,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PID_IL4_2PATHWAY,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0
PID_BCR_5PATHWAY,0.0,0.0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0


In [ ]:
for i in range(plierRes["U"].shape[1]):
    if (mm[i] > 0):
        names[i] = f"LV{i+1}," + ",".join(Uuse.iloc[:,i].sort_values(ascending=False).where(lambda x: x > 0).index[0:top])
    elif: max(plierRes["U"].iloc[, i]) > 0:
        names[i] = f"LV{i+1}," + ",".join(plierRes["U"].iloc[:,i].sort_values(ascending=False).where(lambda x: x > 0).index[0:top])
    else:
        names[i] = f"LV{i+1}"

In [41]:
i = 0

In [42]:
mm[i]

0.0293886583358289

In [49]:
f"LV{i+1}," + Uuse[Uuse.iloc[:, i] == mm[i]].index[0]

'LV1,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY'

In [50]:
f"LV{i+1}," + plierRes["U"][plierRes["U"].iloc[:, i] == mm[i]].index[0]

'LV1,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY'

In [67]:
top = 2

In [76]:
f"LV{i+1}," + ",".join(
    Uuse.iloc[:, i].sort_values(ascending=False).where(lambda x: x > 0).index[0:top]
)

'LV1,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY,IRIS_Bcell-Memory_IgG_IgA'